# TreeEnsembleRegressor and parallelisation

The operator [TreeEnsembleRegressor](https://github.com/onnx/onnx/blob/master/docs/Operators-ml.md#ai.onnx.ml.TreeEnsembleRegressor) describe any tree model (decision tree, random forest, gradient boosting). The runtime is usually implements in C/C++ and uses parallelisation. The notebook studies the impact of the parallelisation.

In [2]:
%matplotlib inline

## Graph

The following graph shows the time ratio between two runtimes depending on the number of observations in a batch (N) and the number of trees in the forest.

In [4]:
from tqdm import tqdm

T = list(range(0, 100000))
for nt in tqdm(T):
    numpy.zeros((150, 50))

100%|██████████| 100000/100000 [00:00<00:00, 164392.90it/s]
